In [1]:
import pandas as pd
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
today = datetime.today().strftime('%Y-%m-%d')
todayminusyear = (datetime.now() - relativedelta(years=1)).strftime('%Y-%m-%d')

In [3]:
try:
    script_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    script_dir = os.getcwd()
base_dir = os.path.dirname(script_dir)
data_dir = os.path.join(base_dir, "Data", "Goose")

In [6]:
songdata = pd.read_csv(os.path.join(data_dir, "songdata.csv"))
venuedata = pd.read_csv(os.path.join(data_dir, "venuedata.csv"))
showdata = pd.read_csv(os.path.join(data_dir, "showdata.csv"))
transitiondata = pd.read_csv(os.path.join(data_dir, "transitiondata.csv"))
setlistdata = pd.read_csv(os.path.join(data_dir, "setlistdata.csv"))

In [7]:
showdata.tail(3)

,show_number,show_id,showdate,showtitle,venue_id,tour_id
700,701,1721059697,2024-12-30,NaN,493,1
701,702,1721059705,2024-12-31,NaN,493,1
702,703,1731950152,2025-02-06,NaN,370,39


In [8]:
last_show = showdata['show_number'].max() - 1
last_show

702

In [9]:
setlist_by_song = pd.merge(setlistdata, showdata, on='show_id', how='left').sort_values(['song_id','show_number']).reset_index(drop=True)
setlist_by_song['gap'] = setlist_by_song.groupby('song_id')['show_number'].diff()
setlist_by_song.loc[setlist_by_song.groupby('song_id').head(1).index, 'gap'] = None 
setlist_by_song.head()

,uniqueid,show_id,song_id,setnumber,position,tracktime,transition_id,isreprise,isjam,footnote,...,jamchart_notes,soundcheck,isverified,isrecommended,show_number,showdate,showtitle,venue_id,tour_id,gap
0,5777,1614315171,1,1,8,NaN,1,0,0,For Kenny.,...,NaN,"Butter Rum, Creatures",0.0,NaN,305,2019-04-11,NaN,93,14,NaN
1,40789,1613690131,400,1,5,NaN,1,0,0,First known version.,...,NaN,NaN,1.0,NaN,4,2014-11-21,NaN,25,1,NaN
2,4912,1613690543,400,1,10,NaN,1,0,0,NaN,...,NaN,NaN,1.0,NaN,6,2014-12-20,NaN,36,13,2.0
3,7612,1616037808,400,1,1,NaN,1,0,0,NaN,...,NaN,NaN,1.0,NaN,14,2015-03-14,NaN,24,1,8.0
4,7591,1616037488,400,2,12,NaN,1,0,0,NaN,...,NaN,NaN,0.0,NaN,16,2015-04-18,NaN,26,1,2.0


In [10]:
my_song_data = (setlist_by_song[setlist_by_song['isreprise'] == 0]
                .merge(songdata[['id', 'song_name', 'isoriginal']], left_on='song_id', right_on='id', how='left').drop(columns=['id'])
                .groupby(['song_name', 'isoriginal'])
                .agg({
                    'show_number': ['count', 'min', 'max'],
                    'gap': ['min', 'max', 'mean', 'median', 'std']
                })
                .reset_index()
                .round(2)
               )

my_song_data.columns = ['_'.join(col).strip() for col in my_song_data.columns.values]

# Rename columns for easier access
my_song_data = my_song_data.rename(columns={
    'song_name_': 'song_name', 
    'isoriginal_': 'is_original',
    'show_number_count': 'times_played_total', 
    'show_number_min': 'debut', 
    'show_number_max': 'last_played', 
    'gap_min': 'min_gap', 
    'gap_max': 'max_gap', 
    'gap_mean': 'avg_gap',
    'gap_median': 'med_gap',  
    'gap_std': 'std_gap'
})

my_song_data['is_original'] = my_song_data['is_original'].astype(int)
my_song_data['current_gap'] = last_show - my_song_data['last_played']

my_song_data = (my_song_data
                .merge(showdata[['show_number', 'showdate']], left_on='debut', right_on='show_number', how='left')
                .rename(columns={'showdate': 'debut_date'}).drop(columns=['show_number', 'debut'])
                .merge(showdata[['show_number', 'showdate']], left_on='last_played', right_on='show_number', how='left')
                .rename(columns={'showdate': 'ltp_date'}).drop(columns=['show_number', 'last_played'])
)[['song_name', 'is_original', 'times_played_total','debut_date','ltp_date','current_gap','avg_gap', 'med_gap', 'std_gap']]
my_song_data['gap_zscore'] = (my_song_data['current_gap'] - my_song_data['avg_gap']) / my_song_data['std_gap']


my_song_data.head()

,song_name,is_original,times_played_total,debut_date,ltp_date,current_gap,avg_gap,med_gap,std_gap,gap_zscore
0,(Marie's The Name) His Latest Flame,0,4,2019-06-28,2022-08-20,188,63.33,81.0,31.47,3.961551
1,(dawn),1,8,2022-01-26,2024-12-13,3,33.86,16.0,45.99,-0.671015
2,(satellite),1,3,2018-06-01,2024-09-13,33,219.50,219.5,200.11,-0.931987
3,2021,0,3,2021-10-08,2021-12-31,241,9.50,9.5,13.44,17.224702
4,726,1,35,2020-06-20,2024-12-13,3,8.79,5.5,7.44,-0.778226


In [11]:
ck_plus = (my_song_data[(my_song_data['is_original'] == 1)&(my_song_data['times_played_total'] > 10)].copy()
           .sort_values(by='gap_zscore', ascending=False).reset_index(drop=True).drop(columns=['is_original','debut_date', 'std_gap','gap_zscore'])
)
ck_plus['current_minus_avg'] = ck_plus['current_gap'] - ck_plus['avg_gap']
ck_plus['current_minus_med'] = ck_plus['current_gap'] - ck_plus['med_gap']

ck_plus.head(25)

,song_name,times_played_total,ltp_date,current_gap,avg_gap,med_gap,current_minus_avg,current_minus_med
0,Honeybee,33,2024-06-10,58,8.62,5.5,49.38,52.5
1,Time to Flee,105,2024-10-24,19,4.59,4.0,14.41,15.0
2,White Lights,72,2024-10-30,15,5.65,5.0,9.35,10.0
3,California Magic,41,2024-11-04,11,5.35,5.0,5.65,6.0
4,Doobie Song,23,2024-09-04,39,16.59,12.0,22.41,27.0
5,Indian River,106,2024-10-30,15,6.49,5.0,8.51,10.0
6,Lead Up,28,2024-11-09,8,5.22,5.0,2.78,3.0
7,Feel it Now,26,2024-11-09,8,5.60,5.0,2.40,3.0
8,Jive I,119,2024-11-07,10,5.83,5.0,4.17,5.0
9,Jive Lee,83,2024-11-07,10,5.62,4.0,4.38,6.0


In [12]:
ricks_notebook_data = (setlist_by_song[(setlist_by_song['isreprise'] == 0)&(setlist_by_song['showdate'] > todayminusyear)]
                .merge(songdata[['id', 'song_name', 'isoriginal']], left_on='song_id', right_on='id', how='left').drop(columns=['id'])
)[['song_name', 'isoriginal', 'show_number', 'showdate','gap']]

ricks_notebook = (ricks_notebook_data.groupby(['song_name', 'isoriginal'])
                  .agg({
                      'show_number': ['count', 'max'],
                      'gap': ['min', 'max', 'mean', 'median', 'std']})
                  .reset_index().round(2)
)

ricks_notebook.columns = ['_'.join(col).strip() for col in ricks_notebook.columns.values]

# Rename columns for easier access
ricks_notebook = ricks_notebook.rename(columns={
    'song_name_': 'song_name', 
    'isoriginal_': 'is_original',
    'show_number_count': 'times_played_in_last_year', 
    'show_number_max': 'last_played', 
    'gap_min': 'min_gap', 
    'gap_max': 'max_gap', 
    'gap_mean': 'avg_gap',
    'gap_median': 'med_gap',  
    'gap_std': 'std_gap'
})

ricks_notebook['is_original'] = ricks_notebook['is_original'].astype(int)
ricks_notebook['current_gap'] = last_show - ricks_notebook['last_played']

ricks_notebook = (ricks_notebook
                .merge(showdata[['show_number', 'showdate']], left_on='last_played', right_on='show_number', how='left')
                .rename(columns={'showdate': 'ltp_date'}).drop(columns=['show_number', 'last_played'])
)[['song_name', 'is_original', 'times_played_in_last_year','ltp_date','current_gap','avg_gap', 'med_gap']]

ricks_notebook = (ricks_notebook[(ricks_notebook['is_original'] == 1)&(ricks_notebook['current_gap'] > 3)]
                  .sort_values(by='times_played_in_last_year', ascending=False)
                  .reset_index(drop=True)
                  .drop(columns=['is_original'])
)

ricks_notebook.head(10)

,song_name,times_played_in_last_year,ltp_date,current_gap,avg_gap,med_gap
0,Turned Clouds,12,2024-11-13,5,6.00,6.0
1,California Magic,11,2024-11-04,11,5.91,6.0
2,Feel it Now,11,2024-11-09,8,5.91,5.0
3,Jive I,10,2024-11-07,10,6.70,6.5
4,Lead Up,10,2024-11-09,8,6.60,6.0
5,Jive Lee,10,2024-11-07,10,6.70,5.5
6,This Old Sea,9,2024-11-12,6,8.00,8.0
7,Indian River,9,2024-10-30,15,7.33,6.0
8,Earthling or Alien?,9,2024-11-12,6,7.33,6.0
9,A Western Sun,9,2024-11-10,7,7.67,7.0


In [13]:
# Saving all datasets to CSV
try:
    script_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    script_dir = os.getcwd()
base_dir = os.path.dirname(script_dir)
save_path = os.path.join(base_dir, "Data", "Goose")
ck_plus.to_csv(os.path.join(save_path, "ck_plus.csv"), index=False)
ricks_notebook.to_csv(os.path.join(save_path, "ricks_notebook.csv"), index=False)